<a href="https://colab.research.google.com/github/abhinavarorags/predict/blob/main/SinkingFund.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Quantlib
import QuantLib as ql

# Bond parameters
issue_date = ql.Date(1, 3, 2024)  # March 1, 2024
maturity_date = ql.Date(1, 3, 2027)  # March 1, 2027
coupon_rate = 0.05  # 5% annual coupon
face_value = 1000000  # $1,000,000 face value
frequency = ql.Semiannual
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count = ql.Thirty360(ql.Thirty360.BondBasis)
settlement_days = 3

# Create bond schedule
schedule = ql.Schedule(issue_date, maturity_date, ql.Period(frequency),
                       calendar, ql.Following, ql.Following,
                       ql.DateGeneration.Backward, False)

# Sinking fund schedule (6 equal payments over 3 years)
sinking_fund_schedule = ql.CallabilitySchedule()
sinking_payment = face_value / 6  # 6 payments over 3 years

sinking_date = issue_date
for i in range(6):
    sinking_date = calendar.advance(sinking_date, 6, ql.Months)
    if sinking_date > maturity_date:
        sinking_date = maturity_date
    callability_price = ql.BondPrice(sinking_payment, ql.BondPrice.Clean)
    sinking_fund_schedule.append(
        ql.Callability(callability_price, ql.Callability.Call, sinking_date)
    )

# Create sinking fund bond
sinking_bond = ql.CallableFixedRateBond(
    settlement_days, face_value, schedule, [coupon_rate],
    day_count, ql.Following, 0.0, issue_date,  # Set redemption to 0
    sinking_fund_schedule
)

# Set pricing engine (Hull-White model + tree)
rate = 0.03
vol = 0.01
term_structure = ql.FlatForward(issue_date, ql.QuoteHandle(ql.SimpleQuote(rate)),
                                day_count, ql.Compounded, frequency)
ts_handle = ql.YieldTermStructureHandle(term_structure)
model = ql.HullWhite(ts_handle, 0.03, vol)
engine = ql.TreeCallableFixedRateBondEngine(model, 40)

# Attach engine to bond
sinking_bond.setPricingEngine(engine)

# Bond price
price = sinking_bond.cleanPrice()
print(f"Bond Price: {price:.2f}")

# Print cash flows
print("\nCash Flows:")
for c in sinking_bond.cashflows():
    print(f"{c.date()} - {c.amount():,.2f}")

# Print sinking fund schedule
print("\nSinking Fund Payments:")
for s in sinking_fund_schedule:
    print(f"{s.date()} - {s.price().amount():,.2f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 41.5 MB/s eta 0:00:00
Bond Price: 9.48

Cash Flows:
September 3rd, 2024 - 25,277.78
March 3rd, 2025 - 25,000.00
September 2nd, 2025 - 24,861.11
March 2nd, 2026 - 25,000.00
September 1st, 2026 - 24,861.11
March 1st, 2027 - 25,000.00
March 1st, 2027 - 0.00

Sinking Fund Payments:
September 3rd, 2024 - 166,666.67
March 3rd, 2025 - 166,666.67
September 3rd, 2025 - 166,666.67
March 3rd, 2026 - 166,666.67
September 3rd, 2026 - 166,666.67
March 1st, 2027 - 166,666.67
